# Geocoding of schools dataset

In [1]:
import pandas as pd
import requests
import os

In [2]:
GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']

In [3]:
folder_path = '../data/raw'
primary_schools = 'mainstream-schools_dublin_2019-2020.xlsx'
secondary_schools = 'post-primary-schools-2019-2020-1.xlsx'
special_schools = 'special-schools_dublin_2019-2020.xlsx'

In [4]:
data = pd.ExcelFile(f"{folder_path}/{primary_schools}")

In [5]:
data.sheet_names

['Sheet1']

In [6]:
df = data.parse('Sheet1')

In [7]:
def extract_lat_long_via_address(address_or_zipcode):
    lat, lng = None, None
    api_key = GOOGLE_API_KEY
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    endpoint = f"{base_url}?address={address_or_zipcode}&key={api_key}"
    # see how our endpoint includes our API key? Yes this is yet another reason to restrict the key
    r = requests.get(endpoint)
    if r.status_code not in range(200, 299):
        return None, None
    try:
        '''
        This try block incase any of our inputs are invalid. This is done instead
        of actually writing out handlers for all kinds of responses.
        '''
        results = r.json()['results'][0]
        lat = results['geometry']['location']['lat']
        lng = results['geometry']['location']['lng']
    except:
        pass
    return (lat, lng)

In [8]:
df['appended_address'] = df.Eircode.fillna(df.iloc[:,1] + ' ' + df.iloc[:,2] + ', ' + df.iloc[:,3] + ', ' + df.iloc[:,4])

In [63]:
print(df['appended_address'])

0      D15P820
1      K78YD27
2      D14YY28
3      D01P027
4      K32PX40
        ...   
447    D6WHP44
448    D18P274
449    D13C3K8
450        NaN
451    D04FP20
Name: appended_address, Length: 452, dtype: object


In [16]:
locations = []

In [17]:
for row in df['appended_address']:
    locations.append(extract_lat_long_via_address(row))

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [36]:
len(locations)

452

In [35]:
for row in df['appended_address'].iloc[len(locations):]:
    locations.append(extract_lat_long_via_address(row))

In [41]:
lat_long = pd.Series(locations, name='lat_long_tuple')

In [42]:
lat_long

0              (53.3750893, -6.3621838)
1              (53.3595512, -6.4435921)
2      (53.2912791, -6.278829399999999)
3              (53.3514878, -6.2560091)
4       (53.57887419999999, -6.2856359)
                     ...               
447            (53.3236468, -6.2776649)
448     (53.27840279999999, -6.2113282)
449            (53.3908101, -6.1131553)
450                        (None, None)
451            (53.3350796, -6.2402099)
Name: lat_long_tuple, Length: 452, dtype: object

In [43]:
lat_series = lat_long[0]

In [57]:
df['latitude'],df['longitude'] = zip(*lat_long)

In [55]:
df.shape

(452, 21)

In [66]:
df.to_csv('../data/processed/schools_primary.csv')